In [45]:
import json
import random
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas(desc='pandas bar')

## CTA task WebTable Dataset

- Check https://github.com/ysunbp/RECA-paper/tree/main/WebTables to download raw-data(k0-k4) for 5-fold validation
- for simplicity, we treat `k3` as test data, and sample 20% of `k0,k1,k2,k4` as training data. The selection is listed in `index` column
- We furthur demonstrate how to merge multiple sub-tables in original file into one dataframe, take `k3` as an example 

In [ ]:
import pandas as pd
import os

def merge_json_files_in_folder(folder_path):
    # 初始化一个空的DataFrame来存放所有数据
    combined_df = pd.DataFrame()

    # 遍历文件夹中的每个文件
    for filename in tqdm(os.listdir(folder_path)):
        file_path = os.path.join(folder_path, filename)
        
        # 确保只处理JSON文件
        if file_path.endswith('.json'):
            # 读取JSON文件
            df = pd.read_json(file_path, lines=True)
            
            # 合并到主DataFrame
            combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    return combined_df

# 使用函数
folder_path = '../raw_data/CTA/WebTable/K3'  # 替换为你的文件夹路径
k3 = merge_json_files_in_folder(folder_path)
def append_content(row):
    id = row[0]
    content = np.array(pd.read_csv('/data/yanmengyi/simpleT5-main/examples/KGC/sato-table/sato_tables/multionly/K3/%s' % id).fillna('').drop_duplicates()).tolist()
    return content
k3['content'] = k3.progress_apply(append_content,axis=1)

In [14]:
webtable_train = pd.read_csv('../raw_dataset/CTA/WebTable/CTA_webtable_train_few_shot.csv',index_col=0)
webtable_test = pd.read_csv('../raw_dataset/CTA/WebTable/CTA_webtable_test.csv',index_col=0)
webtable_test['index'] = webtable_test.index

In [43]:
def Generate_SBERT_Dict_WebTable(row): ## This is for WebTable
    target = int(row['target'])
    label = row['label']
    try:
        content = np.array(eval(row['content']))
    except: ## inf is the occupation format, replace it with empty string
        content = np.array(eval(row['content'].replace('inf',"''")))
    header = eval(row['header'])
    length = content.shape[1]
    text_former = []
    # text_latter = []
    for t in content:
        text = {}
        for l in range(length):
            if(l!=target): ## mask label列
                text[header[l]] = t[l]
            else:
                text['col_1'] = t[l]
        # text += '[SEP]'
        # text += '%s: %s|' % ('col_1',t[target])
        text_former.append(text)
    return text_former
import json
def random_dicts_to_str(input_list, k=5):
    """
    Convert k random dictionaries from a list to JSON strings and join them into a single string separated by newlines.
    
    Parameters:
    input_list (list): The input list containing dictionary elements.
    k (int): The number of dictionaries to randomly select and convert to JSON strings.
    
    Returns:
    str: A string composed of k random JSON strings from the input list, separated by newlines.
    """
    # 确保 k 不大于列表的长度
    k = min(k, len(input_list))
    
    # 随机选择 k 个字典
    selected_dicts = random.sample(input_list, k)
    # 将选中的字典转换为JSON字符串
    if(len(selected_dicts)>1):
        # json_strings = [json.dumps(d, ensure_ascii=True) for d in selected_dicts]
        json_strings = [str(d) for d in selected_dicts]
    else: ## 只有一行
        json_strings = [str(selected_dicts[0])]
    
    # 将JSON字符串以换行符分隔拼接成一个字符串
    
    result_str = '\n'.join(json_strings)
    
    return result_str
def sample_rows_with_all_values_WebTable(df, k):
    """
    Randomly sample k rows from a dataframe, ensuring that the sampled rows include all unique values in the 'pred' column.
    
    Parameters:
    df (pd.DataFrame): The input dataframe.
    k (int): The number of rows to sample.
    
    Returns:
    pd.DataFrame: A dataframe consisting of k randomly sampled rows.
    """
    # 确保k不小于pred列中唯一值的数量
    unique_values = df['label'].unique()
    num_unique_values = len(unique_values)
    k = max(k, num_unique_values)
    
    # 分别从每个唯一值对应的行中随机选取一行
    sampled_rows = [df[df['label'] == val].sample(1) for val in unique_values]
    
    # 如果需要更多的行来达到k，从整个DataFrame中随机选取剩余的行
    additional_rows_needed = k - num_unique_values
    if additional_rows_needed > 0:
        # 从除了已选的行之外的行中随机选择
        remaining_rows = df[~df.index.isin([row.index[0] for row in sampled_rows])]
        additional_sampled_rows = remaining_rows.sample(additional_rows_needed)
        sampled_rows.append(additional_sampled_rows)
    
    # 将所有选中的行合并成一个DataFrame
    sampled_df = pd.concat(sampled_rows)
    
    return sampled_df

In [30]:
dict_list_webtable_train = {}
# for i in tqdm(range(len(webtable_train_list_pd['index'].unique()))):
for i in tqdm(webtable_train['index'].unique()):
    dict_list_webtable_train[i] = Generate_SBERT_Dict_WebTable(webtable_train[webtable_train['index']==i].iloc[0])

  0%|          | 0/14856 [00:00<?, ?it/s]

In [46]:
import random
import json
def RAG_WebTable(row):
    tmp_dict = {}
    index = row['index']
    tmp_dict['type'] = ''
    output_dict = {}
    output_dict['type'] = row['label']
    options = eval(row['pred_list'])
    target_col = eval(row['header'])[row['target']]
    random.shuffle(options)
    text = "You are an expert in relation extraction from wikipedia web table to knowledge graph. Given table title and column pair for Table 1, please choose the most proper type from the provided options. Return in json format.\n\nColumn: %s\n\nOptions:%s\n\nOutput Format Example:\n\n%s\n\nTable 1:\n\n%s\n\nReference tables:\n\n" % ('col_1',json.dumps(options),json.dumps(tmp_dict),random_dicts_to_str(dict_list_webtable_train[index]))
    RAG = ''
    RAG_df = webtable_train[webtable_train['label'].isin(options)]
    RAG_df_sample = sample_rows_with_all_values_WebTable(RAG_df,k=5)
    RAG_df_index = RAG_df_sample['index'].to_list()
    for RAG_tuple_index in RAG_df_index:
        RAG_dict = {}
        RAG_row = RAG_df_sample[RAG_df_sample['index']==RAG_tuple_index].iloc[0] ## 被选择的一个Tuple
        RAG_dict['Table'] = random_dicts_to_str(dict_list_webtable_train[RAG_tuple_index],k=4) ## 
        RAG_dict['Column'] = 'col_1'
        RAG_dict['type'] = RAG_row['label']
        # RAG += '%s\n\n' % json.dumps(RAG_dict, ensure_ascii=False)
        RAG += '%s\n\n' % str(RAG_dict)
    return text + RAG, '', json.dumps(output_dict), index
# print(RAG_WebTable(webtable_train_few_shot.iloc[0])[0])
# print(RAG_WebTable(webtable_train[webtable_train['index']==19493].iloc[0])[0])
webtable_train_output = webtable_train.progress_apply(RAG_WebTable,axis=1,result_type='expand')

pandas bar:   0%|          | 0/14856 [00:00<?, ?it/s]

In [47]:
webtable_train_output.columns = ['instruction','input','output','index']

In [50]:
json.dump(webtable_train_output.to_dict(orient='records'), open('../train/CTA/WebTable/WebTable-train.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [9]:
webtable_test_list = []
for index,row in tqdm(webtable_test.iterrows()):
    query = eval(row['query'])
    for q in query:
        webtable_test_list.append([q,row['label'],row['index']])

0it [00:00, ?it/s]